In [7]:
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, BatchNormalization, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import CosineDecay
import tensorflow as tf
from sklearn.decomposition import TruncatedSVD
from joblib import Parallel, delayed

def extract_features_single(sample, n_components=15):  # 增加n_components
    num_antennas = sample.shape[0]
    reshaped_sample = sample.reshape(num_antennas, -1)
    magnitude = np.abs(reshaped_sample)
    phase = np.angle(reshaped_sample)
    
    # 1. 提取更多SVD特征
    svd_mag = TruncatedSVD(n_components=n_components)
    svd_phase = TruncatedSVD(n_components=n_components)
    magnitude_features = svd_mag.fit_transform(magnitude).flatten()
    phase_features = svd_phase.fit_transform(phase).flatten()
    
    # 2. 增强空间相关性特征
    corr_matrix_mag = np.abs(np.corrcoef(magnitude))
    corr_matrix_phase = np.abs(np.corrcoef(phase))
    spatial_features_mag = corr_matrix_mag[np.triu_indices(corr_matrix_mag.shape[0], k=1)]
    spatial_features_phase = corr_matrix_phase[np.triu_indices(corr_matrix_phase.shape[0], k=1)]
    
    # 3. 统计特征
    mag_stats = np.concatenate([
        np.mean(magnitude, axis=1),
        np.std(magnitude, axis=1),
        np.max(magnitude, axis=1),
        np.min(magnitude, axis=1)
    ])
    
    phase_stats = np.concatenate([
        np.mean(phase, axis=1),
        np.std(phase, axis=1),
        np.max(phase, axis=1),
        np.min(phase, axis=1)
    ])
    
    return np.concatenate([
        magnitude_features,  # 主要SVD特征
        phase_features,     # 主要SVD特征
        spatial_features_mag,  # 空间相关性
        spatial_features_phase,
        mag_stats,     # 辅助统计特征
        phase_stats
    ])

def extract_features(H, n_jobs=-1):
    """Parallel feature extraction for all samples"""
    features = Parallel(n_jobs=n_jobs)(
        delayed(extract_features_single)(sample) 
        for sample in H
    )
    return np.array(features)

def residual_block(x, units):
    """Improved residual block with batch normalization"""
    shortcut = x
    
    # Ensure the shortcut has the same dimensions as the main path
    if int(shortcut.shape[-1]) != units:
        shortcut = Dense(units)(shortcut)
    
    x = BatchNormalization()(x)
    x = Dense(units, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dense(units)(x)
    
    # Add skip connection
    x = tf.keras.layers.Add()([shortcut, x])
    x = tf.keras.layers.Activation('relu')(x)
    
    return x

def build_model(input_dim):
    """Build an improved model with proper residual connections"""
    inputs = Input(shape=(input_dim,))
    
    # Initial dense layer
    x = Dense(128, activation='relu')(inputs)
    x = BatchNormalization()(x)
    
    # Residual blocks
    x = residual_block(x, 128)
    x = residual_block(x, 128)
    
    # Final prediction layers
    x = Dense(64, activation='relu')(x)
    x = BatchNormalization()(x)
    x = Dropout(0.2)(x)
    outputs = Dense(2)(x)
    
    return Model(inputs=inputs, outputs=outputs)

def weighted_mse(y_true, y_pred):
    """更专注于精确定位的损失函数"""
    # 调整权重以更加关注x坐标
    weights = tf.constant([1.2, 0.8])  # x坐标权重更大
    squared_diff = tf.square(y_true - y_pred)
    
    # 添加距离惩罚项
    distance_error = tf.sqrt(tf.reduce_sum(squared_diff, axis=1))
    
    # 组合损失：更注重点到点的精确定位
    return tf.reduce_mean(weights * squared_diff) + 0.1 * tf.reduce_mean(distance_error)

def calcLoc(H, anch_pos, bs_pos, tol_samp_num, anch_samp_num, port_num, ant_num, sc_num):
    # Reshape H to combine port dimensions if needed
    if len(H.shape) == 4:  # If H has shape (samples, antennas, subcarriers, ports)
        H = H.reshape(H.shape[0], H.shape[1], -1)
    
    # Extract features in parallel
    print("Extracting features...")
    features = extract_features(H)
    
    # Prepare training data
    train_indices = anch_pos[:, 0].astype(int) - 1
    X_train = features[train_indices]
    y_train = anch_pos[:, 1:]
    
    # Prepare test data
    test_mask = np.ones(features.shape[0], dtype=bool)
    test_mask[train_indices] = False
    X_test = features[test_mask]
    
    # Standardize features
    scaler_X = StandardScaler()
    X_train_std = scaler_X.fit_transform(X_train)
    X_test_std = scaler_X.transform(X_test)
    
    # Standardize targets
    scaler_y = StandardScaler()
    y_train_std = scaler_y.fit_transform(y_train)
    
    # Split training data
    X_train_split, X_val, y_train_split, y_val = train_test_split(
        X_train_std, y_train_std, test_size=0.15, random_state=42
    )
    
    # Build and compile model
    print("Building and training model...")
    model = build_model(X_train_std.shape[1])
    
    # 使用固定学习率初始化优化器
    initial_lr = 0.001
    optimizer = Adam(learning_rate=initial_lr)
    
    model.compile(
        optimizer=optimizer,
        loss=weighted_mse
    )
    
    # Training callbacks with learning rate reduction
    callbacks = [
        EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True,
            mode='min'
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=7,
            min_lr=1e-6,
            verbose=1
        )
    ]
    
    # Train model
    model.fit(
        X_train_split,
        y_train_split,
        validation_data=(X_val, y_val),
        epochs=200,
        batch_size=32,
        callbacks=callbacks,
        verbose=1
    )
    
    # Predict locations
    print("Making predictions...")
    predictions_std = model.predict(X_test_std)
    predictions = scaler_y.inverse_transform(predictions_std)
    
    # Prepare final results
    final_coords = np.zeros((tol_samp_num, 2))
    final_coords[train_indices] = y_train
    remaining_indices = np.setdiff1d(np.arange(tol_samp_num), train_indices)
    final_coords[remaining_indices] = predictions
    
    return final_coords

In [8]:
import os
import time
import numpy as np
import itertools


# Read in the configuration file
def read_cfg_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()
        line_fmt = [line.rstrip('\n').split(' ') for line in lines]
    info = line_fmt
    bs_pos = list([float(info[0][0]), float(info[0][1]), float(info[0][2])])
    tol_samp_num = int(info[1][0])
    anch_samp_num = int(info[2][0])
    port_num = int(info[3][0])
    ant_num = int(info[4][0])
    sc_num = int(info[5][0])
    return bs_pos, tol_samp_num, anch_samp_num, port_num, ant_num, sc_num

# Read in the info related to the anchor points
def read_anch_file(file_path, anch_samp_num):
    anch_pos = []
    with open(file_path, 'r') as file:
        lines = file.readlines()
        line_fmt = [line.rstrip('\n').split(' ') for line in lines]
    for line in line_fmt:
        tmp = np.array([int(line[0]), float(line[1]), float(line[2])])
        if np.size(anch_pos) == 0:
            anch_pos = tmp
        else:
            anch_pos = np.vstack((anch_pos, tmp))
    return anch_pos

# The channel file is large, read in channels in smaller slices
def read_slice_of_file(file_path, start, end):
    with open(file_path, 'r') as file:
        slice_lines = list(itertools.islice(file, start, end))
    return slice_lines

if __name__ == "__main__":
    # print("<<< Welcome to 2024 Wireless Algorithm Contest! >>>\n")
    ## For ease of data managenment, input data for different rounds are stored in different folders. Feel free to define your own
    # PathSet = {0: "./Test", 1: "./Dataset0", 2: "./CompetitionData2", 3: "./CompetitionData3"}
    # PrefixSet = {0: "Round0", 1: "Round1", 2: "Round2", 3: "Round3"}
    PathSet = {0: "./Test", 1: "./Dataset0", 2: "../Dataset1", 3: "./Dataset2"}
    PrefixSet = {1: "Dataset0", 2: "Dataset1", 3: "Dataset2"}

    Ridx = 2  # Flag defining the round of the competition, used for define where to read data。0:Test; 1: 1st round; 2: 2nd round ...
    PathRaw = PathSet[Ridx]
    Prefix = PrefixSet[Ridx]
    
    ### Get all files in the folder related to the competition. Data for other rounds should be kept in a different folder  
    files = os.listdir(PathRaw)
    names = []
    for f in sorted(files):
        if f.find('CfgData') != -1 and f.endswith('.txt'):
            names.append(f.split('CfgData')[-1].split('.txt')[0])
    
    
    for na in names:
        FileIdx = int(na)
        print('Processing Round ' + str(Ridx - 1) + ' Case ' + str(na))
        
        # Read in the configureation file: RoundYCfgDataX.txt
        # print('Loading configuration data file')
        cfg_path = PathRaw + '/' + Prefix + 'CfgData' + na + '.txt'
        bs_pos, tol_samp_num, anch_samp_num, port_num, ant_num, sc_num = read_cfg_file(cfg_path)
                
        # Read in info related to the anchor points: RoundYInputPosX.txt
        # print('Loading input position file')
        anch_pos_path = PathRaw + '/' + Prefix + 'InputPos' + na + '.txt'
        anch_pos = read_anch_file(anch_pos_path, anch_samp_num)

        # Read in channel data:  RoundYInputDataX.txt
        # slice_samp_num = 1000  # number of samples in each slice
        # slice_num = int(tol_samp_num / slice_samp_num)  # total number of slices
        # csi_path = PathRaw + '/' + Prefix + 'InputData' + na + '.txt'
        # H = []
        # for slice_idx in range(slice_num): # Read in channel data in a loop. In each loop, only one slice of channel is read in
        #     print('Loading input CSI data of slice ' + str(slice_idx))
        #     slice_lines = read_slice_of_file(csi_path, slice_idx * slice_samp_num, (slice_idx + 1) * slice_samp_num)
        #     Htmp = np.loadtxt(slice_lines)
        #     Htmp = np.reshape(Htmp, (slice_samp_num, 2, sc_num, ant_num, port_num))
        #     Htmp = Htmp[:, 0, :, :, :] + 1j * Htmp[:, 1, :, :, :]
        #     Htmp = np.transpose(Htmp, (0, 3, 2, 1))  # Htmp: (slice_samp_num, ant_num, sc_num, port_num)
        #     if np.size(H) == 0:
        #         H = Htmp
        #     else:
        #         H = np.concatenate((H, Htmp), axis=0)
        # H = H.astype(np.complex64) # trunc to complex64 to reduce storage
        
        csi_file = PathRaw + '/' + Prefix + 'InputData' + na + '.npy'
        # np.save(csi_file, H) # After reading the file, you may save txt file into npy, which is faster for python to read 
        H = np.load(csi_file) # if saved in npy, you can load npy file instead of txt
        
        tStart = time.perf_counter()
        
        
        print('Calculating localization results')
        result = calcLoc(H, anch_pos, bs_pos, tol_samp_num, anch_samp_num, port_num, ant_num, sc_num) # This function should be implemented by yourself
        
        # Replace the position information for anchor points with ground true coordinates
        for idx in range(anch_samp_num):
            rowIdx = int(anch_pos[idx][0] - 1)
            result[rowIdx] = np.array([anch_pos[idx][1], anch_pos[idx][2]])

        # Output, be careful with the precision
        print('Writing output position file')
        with open(PathRaw + '/Outputs' + '/' + Prefix + 'Output' + na + '.txt', 'w') as f:
            np.savetxt(f, result, fmt='%.4f %.4f')

        # This help to evaluate the running time, can be removed!
        tEnd = time.perf_counter()
        print("Total time consuming = {}s\n\n".format(round(tEnd - tStart, 3)))

Processing Round 1 Case 1


Calculating localization results
Extracting features...
Building and training model...
Epoch 1/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - loss: 3.1540 - val_loss: 1.1329 - learning_rate: 0.0010
Epoch 2/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.5954 - val_loss: 1.1533 - learning_rate: 0.0010
Epoch 3/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.3384 - val_loss: 1.1360 - learning_rate: 0.0010
Epoch 4/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1982 - val_loss: 1.1549 - learning_rate: 0.0010
Epoch 5/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.1479 - val_loss: 1.1347 - learning_rate: 0.0010
Epoch 6/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.1366 - val_loss: 1.1117 - learning_rate: 0.0010
Epoch 7/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.0111 - val_loss: 1.1212 - learning_rate: 0.0010
Epoch 8/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9858 - val_loss: 1.1436 - learning_rate: 0.0010
Epoch 9/200
54/54 ━━━━━━━━━━━━━━━━━━━━ 0